## Production Amounts and Scope 1 and 2 Emission Values Per EAF Plant in the United States

### Loading Libraries

In [1]:
import pandas as pd
import numpy as np
import janitor
import geopandas as gpd
import mapclassify as mc
from shapely.geometry import Point
import re

pd.options.display.float_format = '{:.5f}'.format

### Importing eGrid Data

The EPA eGrida Data, which was available for 2021 at the time of download, contains the emissions intensity of electricity produced in each sub region. For this analysis, we assume that each steel plant is drawing 100% of its electricity from the grid (and the eGrid subregion that it is located within).

In [2]:
## Importing grid region electricity values
egrid2021_data = pd.read_excel('../data/eGRID2021_data.xlsx', sheet_name = "SRL21").clean_names(strip_underscores = True).drop(0)

### Standardizing state names and cleaning up rows in eGrid data

In [3]:
## Cleaning egrid data for usability
emissions_by_subregion = egrid2021_data.copy()

## Data Processing on columns
emissions_by_subregion["co2e_lbs_per_mwh"] = pd.to_numeric(emissions_by_subregion["egrid_subregion_annual_co2_equivalent_combustion_output_emission_rate_lb_mwh"])
emissions_by_subregion['subregion'] = emissions_by_subregion['egrid_subregion_acronym']
emissions_by_subregion['subregion_name'] = emissions_by_subregion['egrid_subregion_name']
emissions_by_subregion['co2e_tonnes_per_mwh'] = emissions_by_subregion['co2e_lbs_per_mwh'] / 2204.62262185

## Filtering subregions
excluded_subregions = ["AKGD", "AKMS", "HIMS", "HIOA", "PRMS"]
emissions_by_subregion = emissions_by_subregion[~emissions_by_subregion['subregion'].isin(excluded_subregions)]

## Selecting desired columns
emissions_by_subregion = emissions_by_subregion[['subregion', 'subregion_name', 'co2e_tonnes_per_mwh']]

### Importing GEM data

The GEM data was provided by our client Caitlin Swalec and includes the plant names, locations, and plant capacities that we will be reviewing for the EAF steel plants.

In [4]:
gem_data_readin = pd.read_excel("../data/GEM_2022_data.xlsx", sheet_name = "Steel Plants").clean_names(strip_underscores = True)

### Filtering to only look at plants and data that we are interested in

In [5]:
## eaf_capacity is in thousand tonnes per year and we are standardizing to tonnes
gem_data_cleaned = gem_data_readin.copy()

## Filtering to the specifications we need
gem_data_cleaned = gem_data_cleaned[
       (gem_data_cleaned['country'] == "United States") &
       (gem_data_cleaned["status"] == "operating") &
       gem_data_cleaned['nominal_eaf_steel_capacity_ttpa'].notna()
]

## Needed to do this in another step to make sure start_date was properly filtered
gem_data_cleaned = gem_data_cleaned[
       (gem_data_cleaned['start_date'] < 2022) &
       (~gem_data_cleaned['plant_id'].isin(["SUS00009", "SUS00061"]))
]

## Renaming columns
gem_data_cleaned = gem_data_cleaned.rename(columns={'plant_name_english':'plant_name'
                                          , 'subnational_unit_province_state':'state'
                                          , 'location_address':'address'})

## Converting EAF capacity from Thousand Tonnes to Tonnes
gem_data_cleaned['eaf_capacity'] = pd.to_numeric(gem_data_cleaned['nominal_eaf_steel_capacity_ttpa'])
gem_data_cleaned['max_tonnes_of_steel_producible_annually'] = gem_data_cleaned['eaf_capacity'] * 1000

## Reordering columns to desired order
gem_data_cleaned = gem_data_cleaned[['plant_id'
         , 'plant_name'
         , 'owner'
         , 'coordinates'
         , 'country'
         , 'state'
         , 'status'
         , 'start_date'
         , 'plant_age_years'
         , 'max_tonnes_of_steel_producible_annually'
         , 'municipality'
         , 'address'
         , 'category_steel_product'
         , 'steel_products'
         , 'responsiblesteel_certification']]

## Removing columns we do not need
gem_data = gem_data_cleaned.drop(columns=['country', 'start_date', 'status', 'responsiblesteel_certification'])

## Separate the "coordinates" column into "lat" and "lon" columns
gem_data[['lat', 'lon']] = gem_data['coordinates'].str.split(',', expand=True)

## Remove the "coordinates" column
gem_data.drop(columns=['coordinates'], inplace=True)

## Reordering columns
gem_data = gem_data[['plant_id', 'plant_name', 'owner', 'lat', 'lon', 'state', 'plant_age_years', 'max_tonnes_of_steel_producible_annually', 'municipality', 'address', 'category_steel_product', 'steel_products']]

The eGrid data loaded below includes the electric grid subregions that we will be looking at. We are find which points overlap which with regions and are assigning those overlaps as the assigned region.

In [6]:
## Reading in data
subregion_shapes_raw = gpd.read_file("../data/egrid2020_subregions/eGRID2020_subregions.shp").clean_names()

## Filtering subregion shapes
subregion_shapes = subregion_shapes_raw[~subregion_shapes_raw['zipsubregi'].isin(["AKGD", "AKMS", "HIMS", "HIOA", "PRMS"])]

## Simplifying subregion shapes
subregion_shapes['geometry'] = subregion_shapes.simplify(tolerance=0.0005)

## Bringing in plant points
plant_points = gpd.GeoDataFrame(gem_data, geometry=gpd.points_from_xy(gem_data['lon'], gem_data['lat']))
plant_points.crs = "EPSG:4326"

## Finding points that overlap with the egrid subregion
## Removing columns we dont need and renaming the subregion column
plant_emissions_by_subregion = gpd.sjoin(plant_points, subregion_shapes, op='within').drop(columns=['geometry', 'index_right', 'shape_leng', 'shape_le_1', 'shape_area']).rename(columns={'zipsubregi':'subregion'})

plant_emissions_by_subregion = plant_emissions_by_subregion.merge(emissions_by_subregion, on='subregion', how='left')

/Users/mikey/opt/miniconda3/envs/mikeys_env/lib/python3.10/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


### Importing AISI data

The American Iron and Steel Institute (AISI) is a trade association that represents the North American steel industry. It gathers and provides data related to steel production, consumption, trade, and other industry metrics. The AISI data covers various aspects of the steel industry, including information on steel production volumes, capacities, and utilization rates.

We will be using these utilization rates in order to estimate the amount of steel produced per plant per week and per year.

In [7]:
AISI_regions_readin = pd.read_excel("../data/AISI_regions.xlsx", sheet_name="Regions by State").clean_names(strip_underscores = True)

AISI_data_readin = pd.read_excel("../data/AISI_data.xlsx", sheet_name="AISI Production Values").clean_names(strip_underscores = True)

NE = Northeast

GL = Great Lakes

MW = Midwest

S = Southern

W = Western

### Filtering AISI data, renaming columns, and selecting the weekly data and utilization rates we need

**Uilization is based on tonnage capability to produce raw steel for a sustained full order book.**

In [8]:
AISI_regions = AISI_regions_readin[AISI_regions_readin['steel_plant_in_gspt'] == "yes"]

AISI_data = AISI_data_readin.rename(columns = {'north_east_region_capacity_utilization':'NE_util'
                                          , 'great_lakes_region_capacity_utilization':'GL_util'
                                          , 'midwest_region_capacity_utilization':'MW_util'
                                          , 'southern_region_capacity_utilization':'S_util'
                                          , 'western_region_capacity_utilization':'W_util'})

AISI_data = AISI_data[['week_end_date', 'NE_util', 'GL_util', 'MW_util', 'S_util', 'W_util']]

AISI_data = AISI_data[AISI_data['week_end_date'] <= '2022-01-01']

AISI_data['week_end_date'] = pd.to_datetime(AISI_data['week_end_date'])

AISI_data['week_end_date'] = AISI_data['week_end_date'].dt.date

### Merging all of our data so far

In [9]:
regional_plant_emissions = pd.merge(plant_emissions_by_subregion, AISI_regions, on = "state", how = "left")

regional_plant_emissions = regional_plant_emissions.drop(['steel_plant_in_gspt', 'state_abbreviation'], axis = 1)

conditions = [
    regional_plant_emissions['municipality'].isin(["Alton", "Sterling", "Peoria", "Granite City", "Mansfield", "Middletown"])
    , regional_plant_emissions['municipality'].isin(["Riverdale", "Chicago", "Bourbonnais", "Cuyahoga Heights", "Cleveland", "Toledo", "Lorain"])
    , regional_plant_emissions['municipality'].isin(["Mingo Junction", "Youngstown", "Canton"])
]

choices = ["Midwest", "Great Lakes", "North East"]

regional_plant_emissions['region'] = np.select(conditions, choices, default=regional_plant_emissions['region'])

### Defining carbon intensity for eaf steel production and preparing data to calculate intensities

In [10]:
## co2e_tonnes_per_mwh is from eGrid and is pounds of CO2e per MWH of electricity produced per grid location (not regional location)

## Global Efficiency Intelligence states that it takes 710 KWH to produce 1 tonne of steel. "Global Efficiency Intelligence: Industrial Electrification in U.S. States"
## MWH per tonne of steel
eaf_MWH_per_tonne = 710 / 1000

## emissions_intensity is tonnes of CO2e per tonne of steel (the amount of co2e produced for every tonne of steel produced)

AISI_longer = AISI_data.melt(id_vars= 'week_end_date'
                           , value_vars = ['NE_util', 'GL_util', 'MW_util', 'S_util', 'W_util']
                           , var_name = 'region'
                           , value_name = 'utilization')

region_conditions = [
  AISI_longer['region'] == "NE_util"
  , AISI_longer['region'] == "GL_util"
  , AISI_longer['region'] == "MW_util"
  , AISI_longer['region'] == "S_util"
  , AISI_longer['region'] == "W_util"
]

region_choices = ["North East", "Great Lakes", "Midwest", "Southern", "Western"]

AISI_longer['region'] = np.select(region_conditions, region_choices)

### Finding Weekly Scope 2 co2e values and putting it in long and wide format

In [11]:
## Merging our two datasets based on the region column and defining new columns
scope2_plant_emissions_long = pd.merge(regional_plant_emissions, AISI_longer, how = "left", on = "region")

## Creating relavent columns
scope2_plant_emissions_long['estimated_emissions_intensity_tonne_per_tonne_scope2'] = scope2_plant_emissions_long['co2e_tonnes_per_mwh'] * eaf_MWH_per_tonne
scope2_plant_emissions_long['max_tonnes_of_steel_producible_weekly'] = scope2_plant_emissions_long['max_tonnes_of_steel_producible_annually'] / 52
scope2_plant_emissions_long['scope2_co2e_tonnes_per_week'] = scope2_plant_emissions_long['utilization'] * scope2_plant_emissions_long['max_tonnes_of_steel_producible_weekly']

scope2_plant_emissions_long = scope2_plant_emissions_long[["plant_id"
                                                            ,"plant_name"                                          
                                                            ,"owner"                                               
                                                            ,"lat"                                                 
                                                            ,"lon"                                                 
                                                            ,"state"                                               
                                                            ,"plant_age_years"                                     
                                                            ,"municipality"                                        
                                                            ,"address"                                             
                                                            ,"category_steel_product"                              
                                                            ,"steel_products"                                      
                                                            ,"subregion"                                           
                                                            ,"subregion_name"                                      
                                                            ,"co2e_tonnes_per_mwh"                                 
                                                            ,"region"                                              
                                                            ,"week_end_date"                                       
                                                            ,"utilization"                                         
                                                            ,"estimated_emissions_intensity_tonne_per_tonne_scope2"
                                                            ,"max_tonnes_of_steel_producible_annually"             
                                                            ,"max_tonnes_of_steel_producible_weekly"               
                                                            ,"scope2_co2e_tonnes_per_week"]]

## Rounding the data
scope2_plant_emissions_long_rounded = scope2_plant_emissions_long.round(2)

## Transferring each week to be its own column
scope2_plant_emissions_wide = scope2_plant_emissions_long_rounded.drop(columns = ['utilization']).pivot(
    index = ['plant_id', 'plant_name', 'owner', 'lat', 'lon', 'state',
       'plant_age_years', 'municipality', 'address', 'category_steel_product',
       'steel_products', 'subregion', 'subregion_name', 'co2e_tonnes_per_mwh',
       'region',
       'estimated_emissions_intensity_tonne_per_tonne_scope2',
       'max_tonnes_of_steel_producible_annually',
       'max_tonnes_of_steel_producible_weekly']
    , columns = 'week_end_date'
    , values = 'scope2_co2e_tonnes_per_week'
).reset_index(drop=False)


/Users/mikey/opt/miniconda3/envs/mikeys_env/lib/python3.10/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


### Finding Scope 2 values for the complete 2021 year

In [12]:
yearly_scope2 = scope2_plant_emissions_long.groupby(['plant_id', 'plant_name', 'address', 'lat', 'lon']).agg(scope2_co2e_tonnes_2021 = ('scope2_co2e_tonnes_per_week', 'sum')).reset_index()

### Extracting zip codes to find matches with addresses

In [13]:
## Making a copt of yearly_scope2 to work with
zipcode_extraction = yearly_scope2.copy()

## Extracting zip code from address
zipcode_extraction['address2'] = yearly_scope2['address']

## Separate the addresses based off of commas
zipcode_extraction[['zip_code_part1', 'zip_code_part2', 'zip_code_part3', 'zip_code_part4', 'zip_code_part5']] = zipcode_extraction['address2'].str.split(',', expand = True)

## Drop columns we do not need
zipcode_extraction = zipcode_extraction.drop(columns = ['address2', 'zip_code_part1', 'zip_code_part5'])

## Extract only the numeric portions of the addresses and put in NA's if there are no numbers
zipcode_extraction['zip_code_part2'] = zipcode_extraction['zip_code_part2'].str.extract(r'(\d+)')
zipcode_extraction['zip_code_part3'] = zipcode_extraction['zip_code_part3'].str.extract(r'(\d+)')
zipcode_extraction['zip_code_part4'] = zipcode_extraction['zip_code_part4'].str.extract(r'(\d+)')

## Create zip_code column by selecting the non-null values from zip_code_part2, zip_code_part3, zip_code_part4
zipcode_extraction['zip_code'] = zipcode_extraction['zip_code_part2'].fillna(zipcode_extraction['zip_code_part3']).fillna(zipcode_extraction['zip_code_part4'])

## Dropping columns we do not need
zipcode_extraction = zipcode_extraction.drop(columns = ['zip_code_part2', 'zip_code_part3', 'zip_code_part4'])
zipcode_extraction["zip_code"] = pd.to_numeric(zipcode_extraction["zip_code"])

checking_unique_zipcodes_scope2 = zipcode_extraction.groupby('zip_code').size().reset_index(name='count')
checking_unique_zipcodes_scope2 = checking_unique_zipcodes_scope2[checking_unique_zipcodes_scope2['count'] < 2].dropna().reset_index()
checking_unique_zipcodes_scope2 = checking_unique_zipcodes_scope2[['zip_code']]

unique_scope2_zipcodes = pd.merge(checking_unique_zipcodes_scope2, zipcode_extraction, how = "left", on = "zip_code")

### Reading in Greenhouse Gas Emissions data (scope 1) and extracting the non-repeating zip codes

The Greenhouse Gas Emissions data we are using includes emissions data of co2 equivalences from steel plants in the United States. We assume that this data is scope 1 and does not include any scope 2 sources. This data includes steel plant names, companies, locations, and emissions data.

In [14]:
scope1_readin = pd.read_excel("../data/GHG_flight_scope1.xls", skiprows = 5, na_values = ['', 0]).clean_names(strip_underscores = True)

## Making 0 values to NA to align with R code. Python was not reading them in as NA to begin with
scope1_readin['ghg_quantity_metric_tons_co2e'] = scope1_readin['ghg_quantity_metric_tons_co2e'].replace(0, np.nan)

scope1 = scope1_readin[['zip_code', 'ghg_quantity_metric_tons_co2e', 'ghgrp_id']].dropna()

checking_unique_zipcodes_scope1 = scope1.groupby('zip_code').size().reset_index(name = 'count')
checking_unique_zipcodes_scope1 = checking_unique_zipcodes_scope1[checking_unique_zipcodes_scope1['count'] < 2].dropna().reset_index()
checking_unique_zipcodes_scope1 = checking_unique_zipcodes_scope1[['zip_code']]

unique_scope1_zipcodes = pd.merge(checking_unique_zipcodes_scope1, scope1, how = "left", on = "zip_code")

### Joining scope 1 and scope 2 unique zip codes

In [23]:
## Manually checked zip codes and addresses and plant names and the following are for sure a match
scope_1_and_2_zipcode_matches = pd.merge(unique_scope1_zipcodes, unique_scope2_zipcodes, on = "zip_code", how = "inner").dropna().rename(columns = {'ghg_quantity_metric_tons_co2e':'scope1_co2e_tonnes_2021'}).reindex(columns=["zip_code"])

### Anti-Joining to find remaining zip codes that had not matched and manually matching them

In [35]:
## Anti-Joining to find remaining zip codes that had not matched and manually matching them
remaining_zipcodes = pd.merge(zipcode_extraction, scope_1_and_2_zipcode_matches, on = "zip_code", how = "left", indicator = True)
remaining_zipcodes = remaining_zipcodes[remaining_zipcodes['_merge'] == 'left_only'].drop(columns='_merge')

corrected_ghgids = remaining_zipcodes.copy()

## Correctly matching the zip codes and assigning the plant id equivalents between both datasets 
ghgid_conditions = [
  remaining_zipcodes['plant_id'] == "SUS00002"
, remaining_zipcodes['plant_id'] == "SUS00007"
, remaining_zipcodes['plant_id'] == "SUS00015"
, remaining_zipcodes['plant_id'] == "SUS00042"
, remaining_zipcodes['plant_id'] == "SUS00019"
, remaining_zipcodes['plant_id'] == "SUS00025"
, remaining_zipcodes['plant_id'] == "SUS00032"
, remaining_zipcodes['plant_id'] == "SUS00058"
, remaining_zipcodes['plant_id'] == "SUS00029"
]

ghgid_choices = [
  "1003268"
, "1003668"
, "1000394"
, "1004616"
, "1002977"
, "1007642"
, "1007921"
, "1007348"
, "1005700"]

corrected_ghgids['ghgrp_id'] = np.select(ghgid_conditions, ghgid_choices, default=remaining_zipcodes['plant_id'])

corrected_ghgids['ghgrp_id'] = pd.to_numeric(corrected_ghgids['ghgrp_id'])

corrected_ghgids = pd.merge(corrected_ghgids, scope1, how = "left", on = "ghgrp_id").drop(columns = ['zip_code_x', 'zip_code_y']).rename(columns = {'ghg_quantity_metric_tons_co2e':'scope1_co2e_tonnes_2021'})

'\n %>% \n  mutate(ghgrp_id = case_when(plant_id == "SUS00002" ~ "1003268"\n                              , plant_id == "SUS00007" ~ "1003668"\n                              , plant_id == "SUS00015" ~ "1000394"\n                              , plant_id == "SUS00061" ~ "1001699"\n                              , plant_id == "SUS00042" ~ "1004616"\n                              , plant_id == "SUS00019" ~ "1002977"\n                              , plant_id == "SUS00025" ~ "1007642"\n                              , plant_id == "SUS00032" ~ "1007921"\n                              , plant_id == "SUS00058" ~ "1007348"\n                              , plant_id == "SUS00029" ~ "1005700")\n         , ghgrp_id = as.numeric(ghgrp_id)\n         ) %>% \n  left_join(scope1, by = "ghgrp_id") %>% \n  select(-c(zip_code.x, zip_code.y)) %>% \n  rename(scope1_co2e_tonnes_2021 = ghg_quantity_metric_tons_co2e)\n  '

### Combining all of our data to get a final dataset with scope 1 and 2 2021 values

In [36]:
corrected_ghgids

,plant_id,plant_name,address,lat,lon,scope2_co2e_tonnes_2021,ghgrp_id,scope1_co2e_tonnes_2021
0,SUS00002,Cleveland-Cliffs Butler steel plant,"1 Armco Dr, Lyndora, PA 16045-1065, United States",40.845524,-79.921452,721591.62223,1003268,35074.00000
1,SUS00007,Cleveland-Cliffs Coatesville steel plant,"139 Modena Road, Coatesville, PA 19230, United...",39.977819,-75.824512,577273.29778,1003668,149470.00000
2,SUS00015,Gerdau Petersburg Steel Mill,"25801 Hofheimer Way, Petersburg, Virginia 2380...",37.181168,-77.449517,939248.32168,1000394,140045.00000
3,SUS00019,North American Stainless steel plant,"6870 Highway 42 East, Ghent, Kentucky, 41045-9...",38.725418,-85.076015,981950.05785,1002977,352251.00000
4,SUS00025,Nucor Steel Hickman plant,"5929 East State Highway 18, Armorel, Arkansas ...",35.901162,-89.776190,2301158.38811,1007642,454106.00000
5,SUS00029,Nucor Steel Gallatin plant,"4831 U.S. Hwy 42 W, Ghent, Kentucky 41405, Uni...",38.759878,-84.996204,891119.67750,1005700,219010.00000
6,SUS00032,Nucor-Yamato Steel Blytheville plant,"7301 East County Road 142, Blytheville, Arkans...",35.941421,-89.713819,2472101.58266,1007921,469008.00000
7,SUS00042,Steel Dynamics Columbus plant,"1945 Airport Road / P.O. Box 1467, Columbus, M...",33.447580,-88.572683,3193444.29371,1004616,575209.00000
8,SUS00058,Optimus Steel Beaumont plant,"PO BOX 3869, Beaumont, TX 77704, United States",30.08201,-94.074694,657473.82518,1007348,71537.00000
